# Multi-Layer Perceptron

## Import

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

## Code

In [4]:
# Récupération des données en spécifiant le séparateur

TRAIN_FILE_PATH = "./data/ECG200_TRAIN.tsv"
TEST_FILE_PATH = "./data/ECG200_TEST.tsv"

train_df = pd.read_csv(TRAIN_FILE_PATH, delimiter='\t', header=None).dropna()
test_df = pd.read_csv(TEST_FILE_PATH, delimiter='\t', header=None).dropna()


# Séparation en données d'entraînement et données de test

x_train, y_train = train_df.loc[:, '1':], train_df.loc[:, :'0']
x_test, y_test = test_df.loc[:, '1':], test_df.loc[:, :'0']

nb_class = y_train.nunique()